In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

movies_df = pd.read_csv(r'C:\Users\Administrator\Desktop\Studies\Python\Python_Project_Movie_Analysis\Python_Project_Movie_Analysis\0_Dataset\movies_metadata.csv', low_memory= False)

movies_df['release_date'] = pd.to_datetime(movies_df['release_date'],errors='coerce') # Convert data type of release_date to datetime.
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce') # Convert data type of budget to numeric.


def list_name(string_list_dict): # Convert the string list of dictionary into list of name.
    list_dict = ast.literal_eval(string_list_dict) if pd.notna(string_list_dict) else string_list_dict
    return [dict['name'] for dict in list_dict] if isinstance(list_dict,list) else []

movies_df['genres'] = movies_df['genres'].apply(lambda x: list_name(x))
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: list_name(x))
movies_df['production_countries'] = movies_df['production_countries'].apply(lambda x: list_name(x))

movies_df = movies_df.explode('production_countries')
movies_df_US = movies_df[movies_df['production_countries'] == "United States of America"].copy() # Filtering all the movies that was produced in United States.

movies_df_US.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21153 entries, 0 to 45463
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  21153 non-null  object        
 1   belongs_to_collection  2573 non-null   object        
 2   budget                 21153 non-null  float64       
 3   genres                 21153 non-null  object        
 4   homepage               4107 non-null   object        
 5   id                     21153 non-null  object        
 6   imdb_id                21150 non-null  object        
 7   original_language      21149 non-null  object        
 8   original_title         21153 non-null  object        
 9   overview               21116 non-null  object        
 10  popularity             21153 non-null  object        
 11  poster_path            21075 non-null  object        
 12  production_companies   21153 non-null  object        
 13  produc

# Exploratory Data Analysis

## Top-rated Movies (with votes morethan 1000)

1. Filter movies with more than 1000 `value_count`.
2. Sort by `vote_average`.
3. Graph Top_rated movies.


In [ ]:
movies_high_vote = movies_df_US[movies_df_US['vote_count'] > 1000] # Focus on movies with voutcount morethat 1000
movies_high_vote = movies_high_vote[['release_date','title','genres','revenue','vote_count','vote_average']].sort_values('vote_average', ascending= False)



,release_date,title,genres,revenue,vote_count,vote_average
314,1994-09-23,The Shawshank Redemption,"[Drama, Crime]",2.834147e+07,8358.0,8.5
834,1972-03-14,The Godfather,"[Drama, Crime]",2.450664e+08,6024.0,8.5
292,1994-09-10,Pulp Fiction,"[Thriller, Crime]",2.139288e+08,8670.0,8.3
2843,1999-10-15,Fight Club,[Drama],1.008538e+08,9678.0,8.3
1152,1975-11-18,One Flew Over the Cuckoo's Nest,[Drama],1.089813e+08,3001.0,8.3
1178,1974-12-20,The Godfather: Part II,"[Drama, Crime]",4.754284e+07,3418.0,8.3
1184,1984-02-16,Once Upon a Time in America,"[Drama, Crime]",0.000000e+00,1104.0,8.3
522,1993-11-29,Schindler's List,"[Drama, History, War]",3.213656e+08,4436.0,8.3
12481,2008-07-16,The Dark Knight,"[Drama, Action, Crime, Thriller]",1.004558e+09,12269.0,8.3
23673,2014-10-10,Whiplash,[Drama],1.309200e+07,4376.0,8.3


In [ ]:
movies_high_vote.head(10)